In [ ]:
import pandas as pd
import math
import copy

Upload all csv before proceeding

In [ ]:
data = pd.read_csv("dataset.csv")
description = pd.read_csv("symptom_Description.csv")
precaution = pd.read_csv("symptom_precaution.csv")

In [ ]:
data.shape

(4920, 18)

In [ ]:
import numpy as np

# Preparing the dataset

In [ ]:
data = data.replace(np.nan, -1)

In [ ]:
data.head(n=5)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Fungal infection,itching,skin_rash,dischromic _patches,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
description.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [ ]:
precaution.head()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [ ]:
symptoms = set()
for index, row in data.iterrows():
    for sypt in row:
        symptoms.add(sypt)

In [ ]:
try:
    symptoms.remove(-1)
except ValueError:
    pass

In [ ]:
sympts = copy.deepcopy(list(symptoms)) 
sympts = [val.strip() for val in sympts]

In [ ]:
len(sympts)

172

In [ ]:
df = pd.DataFrame(columns=sympts)

In [ ]:
import numpy as np

In [ ]:
dic = {"Name":[], "Symptoms":[]}
for index, row in data.iterrows():
    arr = np.zeros(172)
    for each_sympt in row[1:]:
        if each_sympt != -1:
            arr[sympts.index(each_sympt.strip())] = 1
    dic["Name"].append(row[0])
    dic["Symptoms"].append(arr)

In [ ]:
df = pd.DataFrame(dic)

In [ ]:
split_df = pd.DataFrame(df['Symptoms'].tolist(), columns=sympts)

In [ ]:
df = df.join(split_df)

In [ ]:
df = df.drop(columns = ["Symptoms"])

What I have done is that I have taken all the disease and put a 1 if that symptom is that for that disease and 0 if that symptoms i not present in that disease

In [ ]:
df.head(n=15)

,Name,Fungal infection,skin_peeling,Drug Reaction,Diabetes,irritability,dark_urine,continuous_sneezing,pain_behind_the_eyes,visual_disturbances,family_history,cold_hands_and_feets,pus_filled_pimples,lack_of_concentration,stiff_neck,shivering,mucoid_sputum,receiving_blood_transfusion,nausea,itching,Chicken pox,joint_pain,polyuria,extra_marital_contacts,irregular_sugar_level,movement_stiffness,hip_joint_pain,skin_rash,acidity,stomach_pain,headache,congestion,increased_appetite,Hyperthyroidism,sinus_pressure,Alcoholic hepatitis,slurred_speech,Paralysis (brain hemorrhage),Gastroenteritis,GERD,...,yellow_urine,unsteadiness,lethargy,(vertigo) Paroymsal Positional Vertigo,diarrhoea,fluid_overload,cough,swelled_lymph_nodes,red_spots_over_body,muscle_weakness,internal_itching,rusty_sputum,vomiting,loss_of_balance,spinning_movements,enlarged_thyroid,blackheads,coma,inflammatory_nails,Migraine,obesity,yellowish_skin,weakness_in_limbs,AIDS,loss_of_appetite,excessive_hunger,silver_like_dusting,neck_pain,mild_fever,Pneumonia,abnormal_menstruation,bruising,Varicose veins,history_of_alcohol_consumption,Peptic ulcer diseae,bladder_discomfort,receiving_unsterile_injections,Hypothyroidism,indigestion,fast_heart_rate
0,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Fungal infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# Making the decesion tree 


This function is for entropy calculations

In [ ]:
def cal_entropy(df):
    dic = {}
    entropy = df["Name"]
    for index, row in entropy.iteritems():
        dic[row] = dic.get(row, 0) + 1
    total = sum(dic.values())
    H = 0
    for key, values in dic.items():
        Pc = values/total
        H += -Pc * math.log2(Pc)
    return H

This function is for Information Gain

In [ ]:
def IG(entropy, entropy1, entropy2, l1, l2, total):
    return entropy - (l1/total)*entropy1 - (l2/total)*entropy2

This is the decesion tree function. 

In [ ]:
def decesion_tree(tdf, sym):
    if tdf.Name.nunique() <= 1:
        return tdf.Name.unique()
    else:
        max_ig = 0
        split = None
        no = None
        yes = None
        for every_col in sym:
            entropy = cal_entropy(tdf)
            table_1 = tdf[tdf[every_col] == 0.0]
            table_2 = tdf[tdf[every_col] == 1.0]
            entropy1 = cal_entropy(table_1)
            entropy2 = cal_entropy(table_2)
            Ig = IG(entropy, entropy1, entropy2, len(table_1), len(table_2), len(tdf))
            if Ig > max_ig:
                split = every_col
                max_ig = Ig
                no = table_1
                yes = table_2
        user_input = input(f"Do you have {split}. Enter 0 for no, Enter 1 for Yes ")
        sym.remove(split)
        if user_input == "1":
            return decesion_tree(yes.drop(columns=[split]), sym)
        else:
            return decesion_tree(no.drop(columns=[split]), sym)

In [ ]:
arr = decesion_tree(df, sympts[:])

172
Do you have fatigue. Enter 0 for no, Enter 1 for Yes 1
Do you have loss_of_appetite. Enter 0 for no, Enter 1 for Yes 0
Do you have high_fever. Enter 0 for no, Enter 1 for Yes 1
Do you have chest_pain. Enter 0 for no, Enter 1 for Yes 1
Do you have congestion. Enter 0 for no, Enter 1 for Yes 1


In [ ]:
print("You are suffering from", arr[0])
print("Symptom-Description: ", description[description["Disease"] == arr[0]]["Description"].tolist()[0])
temp_df = precaution[precaution["Disease"] == arr[0]]
precautions = []
for index, row in temp_df.iterrows():
        for each_item in row[1:]:
            precautions.append(each_item)
print("Precautions that can be taken =>")
for i in range(len(precautions)):
    print(i+1, precautions[i])

You are suffering from Common Cold
Symptom-Description:  The common cold is a viral infection of your nose and throat (upper respiratory tract). It's usually harmless, although it might not feel that way. Many types of viruses can cause a common cold.
Precautions that can be taken =>
1 drink vitamin c rich drinks
2 take vapour
3 avoid cold food
4 keep fever in check


# The above version is tedious because you have to click 0-1 for every symptoms that may not be relevant too.One More Version where you just have to type out all the symptoms at once you have

In [ ]:
li = input("Enter all the sympts you have seperated by space ").split()

Enter all the sympts you have seperated by space cough high_fever


In [ ]:
# Cleaning the dataset that the user inputted
li = [word.lower() for word in li]

In [ ]:
li

['cough', 'high_fever']

In [ ]:
def decesion_tree_v2(tdf, sym, li):
    if tdf.Name.nunique() <= 10 or len(li) == 0:
        return tdf.Name
    else:
        max_ig = 0
        split = None
        no = None
        yes = None
        for every_col in sym:
            entropy = cal_entropy(tdf)
            table_1 = tdf[tdf[every_col] == 0.0]
            table_2 = tdf[tdf[every_col] == 1.0]
            entropy1 = cal_entropy(table_1)
            entropy2 = cal_entropy(table_2)
            Ig = IG(entropy, entropy1, entropy2, len(table_1), len(table_2), len(tdf))
            if Ig > max_ig:
                split = every_col
                max_ig = Ig
                no = table_1
                yes = table_2
        sym.remove(split)
        if split in li:
            li.remove(split)
            return decesion_tree_v2(yes.drop(columns=[split]), sym, li)
        else:
            return decesion_tree_v2(no.drop(columns=[split]), sym, li)

In [ ]:
arr = decesion_tree_v2(df, sympts[:], li[:])

In [ ]:
disease = {}
for each_item in arr:
    disease[each_item] = disease.get(each_item, 0) + 1

In [ ]:
probable_diesase = []
total = sum(disease.values())
for key, values in disease.items():
    probable_diesase.append((key, values/total))
probable_diesase.sort(key=lambda x: x[1])

In [ ]:
print("Based on the information you provided, You can have")
if len(probable_diesase) > 1:
    for items in probable_diesase:
        print(items[0])
else:
    print(probable_diesase[0][0])

Based on the information you provided, You can have
Pneumonia
Impetigo
Bronchial Asthma
AIDS


In [ ]:
user_input = None

if len(probable_diesase)>1:
    user_input = input("We can't filter out a single disease based on the information you provided so do you want to dig deep to get accurate results(Yes/No)")

We can't filter out a single disease based on the information you provided so do you want to dig deep to get accurate results(Yes/No)Yes


In [ ]:
if user_input is not None and user_input == "Yes":
    all_disease = disease.keys()
    new_df = df.loc[df['Name'].isin(all_disease)]
    arr = decesion_tree(new_df, sympts[:])  
    print("You are suffering from", arr[0])
    print("Symptom-Description: ", description[description["Disease"] == arr[0].strip()]["Description"].tolist()[0])
    temp_df = precaution[precaution["Disease"] == arr[0]]
    precautions = []
    for index, row in temp_df.iterrows():
            for each_item in row[1:]:
                precautions.append(each_item)
    print("Precautions that can be taken =>")
    for i in range(len(precautions)):
        print(i+1, precautions[i])  

Do you have breathlessness. Enter 0 for no, Enter 1 for Yes 1
Do you have chest_pain. Enter 0 for no, Enter 1 for Yes 1
You are suffering from Pneumonia
Symptom-Description:  Pneumonia is an infection in one or both lungs. Bacteria, viruses, and fungi cause it. The infection causes inflammation in the air sacs in your lungs, which are called alveoli. The alveoli fill with fluid or pus, making it difficult to breathe.
Precautions that can be taken =>
1 consult doctor
2 medication
3 rest
4 follow up


In [ ]:
1

1